# Point Cloud Visualization and Manipulation

In [1]:
import vtk
from vtk.util import numpy_support
import numpy as np
from vtk.util.numpy_support import vtk_to_numpy as vtk2np
from vtk.util.numpy_support import numpy_to_vtk as np2vtk

In [2]:
# The following code has been written by normanius under the CC BY-SA 4.0 
# license.
# License:    https://creativecommons.org/licenses/by-sa/4.0/
# Author:     normanius: https://stackoverflow.com/users/3388962/normanius
# Date:       October 2018
# Reference:  https://stackoverflow.com/questions/52716438
###Tolerance = 1e-10 

def fitPlane(X, tolerance=1e-10):
    '''
    Estimate the plane normal by means of ordinary least squares.
    Requirement: points X span the full column rank. If the points lie in a
    perfect plane, the regression problem is ill-conditioned!

    Formulas:
        a = (XX^T)^(-1)*X*z
    Surface normal:
        n = [a[0], a[1], -1]
        n = n/norm(n)
    Plane intercept:
        c = a[2]/norm(n)

    NOTE: The condition number for the pseudo-inverse improves if the
          formulation is changed to homogenous notation.
    Formulas (homogenous):
        a = (XX^T)^(-1)*[1,1,1]^T
        n = a[:-1]
        n = n/norm(n)
        c = a[-1]/norm(n)

    Arguments:
        X:          A matrix with n rows and 3 columns
        tolerance:  Minimal condition number accepted. If the condition
                    number is lower, the algorithm returns None.

    Returns:
        If the computation was successful, a numpy array of length three is
        returned that represents the estimated plane normal. On failure,
        None is returned.
    '''
    X = np.asarray(X)
    d,N = X.shape
    X = np.vstack([X,np.ones([1,N])])
    z = np.ones([d+1,1])
    XXT = np.dot(X, np.transpose(X)) # XXT=X*X^T
    if np.linalg.det(XXT) < 1e-10:
        # The test covers the case where n<3
        return None
    n = np.dot(np.linalg.inv(XXT), z)
    intercept = n[-1]
    n = n[:-1]
    scale = np.linalg.norm(n)
    n /= scale
    intercept /= scale
    return n

def findTransformFromVectors(originFrom=None, axisFrom=None,
                             originTo=None, axisTo=None,
                             origin=None,
                             scale=1):
    '''
    Compute a transformation that maps originFrom and axisFrom to originTo
    and axisTo respectively. If scale is set to 'auto', the scale will be
    determined such that the axes will also match in length:
        scale = norm(axisTo)/norm(axisFrom)

    Arguments:  originFrom:     sequences with 3 elements, or None
                axisFrom:       sequences with 3 elements, or None
                originTo:       sequences with 3 elements, or None
                axisTo:         sequences with 3 elements, or None
                origin:         sequences with 3 elements, or None,
                                overrides originFrom and originTo if set
                scale:          - scalar (isotropic scaling)
                                - sequence with 3 elements (anisotropic scaling),
                                - 'auto' (sets scale such that input axes match
                                  in length after transforming axisFrom)
                                - None (no scaling)

    Align two axes alone, assuming that we sit on (0,0,0)
        findTransformFromVectors(axisFrom=a0, axisTo=a1)
    Align two axes in one point (all calls are equivalent):
        findTransformFromVectors(origin=o, axisFrom=a0, axisTo=a1)
        findTransformFromVectors(originFrom=o, axisFrom=a0, axisTo=a1)
        findTransformFromVectors(axisFrom=a0, originTo=o, axisTo=a1)
    Move between two points:
        findTransformFromVectors(orgin=o0, originTo=o1)
    Move from one position to the other and align axes:
        findTransformFromVectors(orgin=o0, axisFrom=a0, originTo=o1, axisTo=a1)
    '''
    # Prelude with trickle-down logic.
    # Infer the origins if an information is not set.
    if origin is not None:
        # Check for ambiguous input.
        assert(originFrom is None and originTo is None)
        originFrom = origin
        originTo = origin
    if originFrom is None:
        originFrom = originTo
    if originTo is None:
        originTo = originFrom
    if originTo is None:
        # We arrive here only if no origin information was set.
        originTo = [0.,0.,0.]
        originFrom = [0.,0.,0.]
    originFrom = np.asarray(originFrom)
    originTo = np.asarray(originTo)
    # Check if any rotation will be involved.
    axisFrom = np.asarray(axisFrom)
    axisTo = np.asarray(axisTo)
    axisFromL2 = np.linalg.norm(axisFrom)
    axisToL2 = np.linalg.norm(axisTo)
    if axisFrom is None or axisTo is None or axisFromL2==0 or axisToL2==0:
        rotate = False
    else:
        rotate = not np.array_equal(axisFrom, axisTo)
    # Scale.
    if scale is None:
        scale = 1.
    if scale == 'auto':
        scale = axisToL2/axisFromL2 if axisFromL2!=0. else 1.
    if np.isscalar(scale):
        scale = scale*np.ones(3)
    if rotate:
        rAxis = np.cross(axisFrom.ravel(), axisTo.ravel())  # rotation axis
        angle = np.dot(axisFrom, axisTo) / axisFromL2 / axisToL2
        angle = np.arccos(angle)

    # Here we finally compute the transform.
    trafo = vtk.vtkTransform()
    trafo.Translate(originTo)
    if rotate:
        trafo.RotateWXYZ(angle / np.pi * 180, rAxis[0], rAxis[1], rAxis[2])
    trafo.Scale(scale[0],scale[1],scale[2])
    trafo.Translate(-originFrom)
    return trafo

In [3]:
filename = 'DFW_Early_190705_group1_densified_point_cloud.ply'

f = filename#.replace('_group1_densified_point_cloud.ply', '')

r = vtk.vtkPLYReader()
r.SetFileName(filename)
r.Update()

vgf = vtk.vtkVertexGlyphFilter()
vgf.SetInputConnection(r.GetOutputPort())
vgf.Update()

# Generate a Grayscale Lookup Table for the colors to be used in the height map. 
lut = vtk.vtkLookupTable()
lut.SetHueRange(0, 0)
lut.SetSaturationRange(0, 0)
lut.SetValueRange(0.2, 1)
lut.Build()

# Generate a Colored Lookup Table for the colors to be used in the height map.
lut2 = vtk.vtkLookupTable()
lut2.SetHueRange(0.667, 0)
lut2.SetSaturationRange(1, 1)
lut2.SetValueRange(1, 1)
lut2.Build()

print('Number of Colors:',lut.GetNumberOfColors())
print('Number of Colors:',lut2.GetNumberOfColors())

Number of Colors: 256
Number of Colors: 256


In [4]:
#Set the transformation parameters based on input cloud points
source = vgf.GetOutput()

points = vtk2np(source.GetPoints().GetData())
points = points.transpose()

nRegression = fitPlane(points)

trafo = findTransformFromVectors(axisFrom=nRegression.transpose(), originTo=(0,0,0), axisTo=(0.,0.,1.))

In [5]:
#Apply the falttening transformation
sourceTransformed = vtk.vtkTransformFilter()
sourceTransformed.SetInputData(source)
sourceTransformed.SetTransform(trafo)
sourceTransformed.Update()

In [10]:
#Set the elevation filter, applying the color to the height map Pre-Algorithm
pc_bf = vgf.GetOutput() #Before Flattening
bounds_bf = pc_bf.GetBounds()
print(bounds_bf)
minz = bounds_bf[4]
maxz = bounds_bf[5]
print(bounds_bf[4], bounds_bf[5])
lut.SetTableRange(minz, maxz)
lut2.SetTableRange(minz, maxz)
ele_bf = vtk.vtkElevationFilter()
ele_bf.SetInputConnection(vgf.GetOutputPort()) #Before Flattening
ele_bf.SetLowPoint(0, 0, minz)
ele_bf.SetHighPoint(0, 0, maxz)
ele_bf.Update()
pc_bf.GetNumberOfPoints()
pc_bf.GetCenter()

(-35.120819091796875, 38.74599075317383, -20.70392608642578, 21.752647399902344, 47.504615783691406, 49.13410186767578)
47.504615783691406 49.13410186767578


(1.8125858306884766, 0.5243606567382812, 48.319358825683594)

In [6]:
#Set the elevation filter, applying the color to the height map Post-Algorithm
pc = sourceTransformed.GetOutput() #Flattened Point Cloud
bounds = pc.GetBounds()
print(bounds)
minz = bounds[4]
maxz = bounds[5]
print(bounds[4], bounds[5])
lut.SetTableRange(minz, maxz)
lut2.SetTableRange(minz, maxz)
ele = vtk.vtkElevationFilter()
ele.SetInputConnection(sourceTransformed.GetOutputPort()) #Flattened Point Cloud
ele.SetLowPoint(0, 0, minz)
ele.SetHighPoint(0, 0, maxz)
ele.Update()
pc.GetNumberOfPoints()
pc.GetCenter()

#N.B. If Z values have flipped and become negative (when compared to Pre-algorithm values) then use the before flattening (bf) outputs for visualisation

(-37.037818908691406, 40.4487190246582, -26.665889739990234, 24.462724685668945, -49.1235466003418, -47.49711227416992)
-49.1235466003418 -47.49711227416992


(1.7054500579833984, -1.1015825271606445, -48.31032943725586)

In [8]:
# Visualization method Grayscale Pre-Algorithm.

mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(ele_bf.GetOutputPort())
mapper.SetLookupTable(lut)

actor = vtk.vtkActor()
actor.SetMapper(mapper)

renderer = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(renderer)
renWin.SetFullScreen(1)

iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

renderer.AddActor(actor)
renderer.SetBackground(0,0,0)

renWin.Render()

rendLrg = vtk.vtkRenderLargeImage()
writer = vtk.vtkPNGWriter()
rendLrg.SetInput(renderer)
writer.SetInputConnection(rendLrg.GetOutputPort())
writer.SetFileName(f + '_Pre-Algorithm.png')
writer.Write()

iren.Start()

In [9]:
# Visualization method Grayscale Post-Algorithm.

mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(ele.GetOutputPort())
mapper.SetLookupTable(lut)

actor = vtk.vtkActor()
actor.SetMapper(mapper)

renderer = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(renderer)
renWin.SetFullScreen(1)

iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

renderer.AddActor(actor)
renderer.SetBackground(0,0,0)

renWin.Render()

rendLrg = vtk.vtkRenderLargeImage()
writer = vtk.vtkPNGWriter()
rendLrg.SetInput(renderer)
writer.SetInputConnection(rendLrg.GetOutputPort())
writer.SetFileName(f + 'Height_Map.png')
writer.Write()

iren.Start()

In [9]:
# Visualization method Colored Pre-Algorithm.

mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(ele_bf.GetOutputPort())
mapper.SetLookupTable(lut2)

actor = vtk.vtkActor()
actor.SetMapper(mapper)

renderer = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(renderer)
renWin.SetFullScreen(1)

iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

renderer.AddActor(actor)
renderer.SetBackground(0,0,0)

renWin.Render()

rendLrg = vtk.vtkRenderLargeImage()
writer = vtk.vtkPNGWriter()
rendLrg.SetInput(renderer)
writer.SetInputConnection(rendLrg.GetOutputPort())
writer.SetFileName(f + '_Pre-Algorithm_Colored.png')
writer.Write()

iren.Start()

In [9]:
# Visualization method Colored Post-Algorithm.

mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(ele.GetOutputPort())
mapper.SetLookupTable(lut2)

actor = vtk.vtkActor()
actor.SetMapper(mapper)

renderer = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(renderer)
renWin.SetFullScreen(1)

iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

renderer.AddActor(actor)
renderer.SetBackground(0,0,0)

renWin.Render()

rendLrg = vtk.vtkRenderLargeImage()
writer = vtk.vtkPNGWriter()
rendLrg.SetInput(renderer)
writer.SetInputConnection(rendLrg.GetOutputPort())
writer.SetFileName(f + 'Height_Map_Colored.png')
writer.Write()

iren.Start()